# Non-Ideal Reactor Design with Residence Time Distribution (RTD)

**Objective:** This lesson delves into the analysis of non-ideal reactors. We will explore how real reactors deviate from ideal CSTR and PFR behavior and learn how to characterize this non-ideal flow using Residence Time Distribution (RTD). Ultimately, we will use this RTD data to predict the reactor's performance.

**Learning Goals:**
1.  Understand the concepts of non-ideal flow, including dead zones and bypassing/channeling.
2.  Learn how a tracer experiment is used to generate an RTD curve ($E(t)$).
3.  Derive and plot the theoretical RTD curves for ideal PFR and CSTR reactors.
4.  Model a non-ideal reactor using the **Tanks-in-Series (TIS)** model and see how it bridges the gap between CSTRs and PFRs.
5.  Use the **Segregated Flow Model** to predict the conversion of a chemical reaction in a non-ideal reactor.

## Part 1: Why Ideal Reactors Aren't Real

The CSTR and PFR models are mathematical ideals based on strict assumptions about mixing:
*   **Ideal CSTR:** Assumes mixing is instantaneous and complete.
*   **Ideal PFR:** Assumes no mixing in the axial direction, but perfect mixing radially.

In reality, industrial reactors suffer from non-ideal flow patterns:
*   **Dead Zones:** Regions of the reactor with very low flow, where fluid gets 'stuck'.
*   **Bypassing or Channeling:** When fluid takes a short-cut from the inlet to the outlet, spending very little time in the reactor.

These non-idealities mean that not all fluid elements spend the same amount of time in the reactor. The **Residence Time Distribution (RTD)** is a probability distribution that describes the amount of time that fluid elements spend inside the reactor.

## Part 2: The RTD Experiment and the E-Curve

To measure the RTD, we perform a **tracer experiment**. We introduce an inert chemical (a tracer) at the inlet and measure its concentration, $C(t)$, at the outlet over time. For a pulse input, the resulting $C(t)$ curve might look like a skewed bell curve.

From this, we define the **Residence Time Distribution function, $E(t)$**, also called the **E-curve**. It is the normalized concentration curve:
$$ E(t) = \frac{C(t)}{\int_0^\infty C(t) \, dt} $$
$E(t)dt$ represents the fraction of fluid leaving the reactor that has spent a time between $t$ and $t+dt$ inside the reactor. It has two key properties:
$$ \int_0^\infty E(t) \, dt = 1 $$
$$ \int_0^\infty t \cdot E(t) \, dt = \tau  $$
 (The mean of the distribution is the space time) 

## Part 3: RTD of Ideal Reactors - Our Baseline

To understand non-ideal behavior, we must first know what ideal behavior looks like in terms of RTD.

**1. Ideal PFR:** Every fluid element moves together like a plug. They all enter at $t=0$ and all exit at exactly $t=\tau$. The RTD is a perfect Dirac delta function at $t=\tau$.

**2. Ideal CSTR:** The feed is instantly mixed. Some fluid elements will leave almost immediately, while some will linger for a very long time. The RTD can be derived from a mass balance on the tracer:
$$ E(t) = \frac{1}{\tau} e^{-t/\tau} $$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp, quad
from scipy.special import gamma # For the factorial function

# --- Define System Parameters ---
tau = 10.0      # Mean residence time (minutes)

# Time vector for plotting
t = np.linspace(0, 4*tau, 500)

# --- Theoretical E-Curves ---
# CSTR
E_cstr = (1/tau) * np.exp(-t/tau)

# PFR (approximated as a tall, thin pulse for visualization)
E_pfr = np.zeros_like(t)
pfr_pulse_index = np.argmin(np.abs(t - tau))
E_pfr[pfr_pulse_index] = 50 # Arbitrary height for visibility

# --- Plotting the Ideal E-Curves ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(12, 7))
plt.plot(t, E_cstr, 'r-', linewidth=3, label='Ideal CSTR')
plt.plot(t, E_pfr, 'b-', linewidth=3, label='Ideal PFR (visualized pulse)')
plt.title('Theoretical RTD for Ideal Reactors', fontsize=16, weight='bold')
plt.xlabel('Time, t (minutes)', fontsize=14)
plt.ylabel('E(t) (1/min)', fontsize=14)
plt.legend()
plt.show()

## Part 4: The Tanks-in-Series (TIS) Model for Non-Ideal Reactors

This is our first real model for non-ideal flow. It approximates a non-ideal reactor as a cascade of $N$ equal-sized ideal CSTRs. 

*   If $N=1$, we have a perfect CSTR.
*   If $N \rightarrow \infty$, the cascade approaches a perfect PFR.
*   For intermediate values ($1 < N < \infty$), the model represents a reactor with some degree of dispersion or back-mixing, which is very realistic.

The E-curve for the TIS model is:
$$ E(t) = \frac{N}{\tau} \frac{(Nt/\tau)^{N-1}}{(N-1)!} e^{-Nt/\tau} $$

In [ ]:
def e_curve_tis(t, tau, N):
    """E-Curve for the Tanks-in-Series model."""
    # We use the gamma function since Gamma(N) = (N-1)!
    # This is more robust and works for non-integer N.
    return (N/tau) * (N*t/tau)**(N-1) / gamma(N) * np.exp(-N*t/tau)

# --- Plotting TIS E-Curves for different N ---
plt.figure(figsize=(12, 7))

N_values = [1, 2, 5, 10, 50]
for N in N_values:
    E_tis = e_curve_tis(t, tau, N)
    plt.plot(t, E_tis, label=f'Tanks-in-Series, N={N}')

plt.title('Tanks-in-Series RTD Model', fontsize=16, weight='bold')
plt.xlabel('Time, t (minutes)', fontsize=14)
plt.ylabel('E(t) (1/min)', fontsize=14)
plt.legend()
plt.show()

print("Observe how as N increases, the curve becomes narrower and more symmetric, approaching a PFR's pulse.")

## Part 5: Using RTD to Predict Reactor Conversion

Knowing the RTD is great, but its real power is in predicting the performance of a chemical reaction. The **Segregated Flow Model** is one way to do this.

**The Concept:** Imagine the fluid flowing through the reactor as a collection of tiny, separate "packets". Each packet is a miniature batch reactor. The final outlet stream is the mixture of all these packets, which have all spent different amounts of time in the reactor.

The overall conversion ($X_A$) is the average of the conversions achieved in each individual packet, weighted by the fraction of packets that spent that amount of time in the reactor (which is exactly what $E(t)$ tells us!).
$$ \bar{X}_A = \int_0^\infty X_{A,batch}(t) \cdot E(t) \, dt $$
where $X_{A,batch}(t)$ is the conversion achieved in a batch reactor after time $t$.

In [ ]:
# --- Define the reaction and batch kinetics ---
k = 0.25 # Rate constant for a first-order reaction (1/min)

def batch_conversion(t, k):
    """Conversion for a first-order reaction in a batch reactor."""
    return 1 - np.exp(-k * t)

# --- Calculate Conversion for the TIS model ---

# Let's choose a specific non-ideal reactor, e.g., N=3
N_non_ideal = 3

# The function to be integrated (the integrand)
def integrand(t, k, tau, N):
    return batch_conversion(t, k) * e_curve_tis(t, tau, N)

# Perform the numerical integration from t=0 to t=infinity
conversion_non_ideal, error = quad(integrand, 0, np.inf, args=(k, tau, N_non_ideal))

# --- For comparison, calculate conversion in ideal reactors ---
conversion_pfr = batch_conversion(tau, k)
conversion_cstr = (k * tau) / (1 + k * tau)

print(f"--- Predicted Conversion for tau = {tau} min ---")
print(f"Ideal PFR:       {conversion_pfr:.3f}")
print(f"Non-Ideal (N={N_non_ideal}): {conversion_non_ideal:.3f}")
print(f"Ideal CSTR:      {conversion_cstr:.3f}")

## Conclusion and Challenges

We have successfully characterized a non-ideal reactor using the Tanks-in-Series model and used its RTD to predict a conversion that falls predictably between the ideal PFR and CSTR cases. This demonstrates the power of RTD analysis in providing a more realistic picture of reactor performance.

### Student Challenges

1.  **Effect of Non-Ideality:** How does the predicted conversion change as you vary `N_non_ideal` from 2 up to 20? Create a plot of Conversion vs. N. What does this tell you about the importance of preventing back-mixing in reactor design?
2.  **Effect of Reaction Order:** The Segregated Flow Model gives different results for different reaction orders. Modify the `batch_conversion` function for a **second-order reaction**, where $X_A(t) = \frac{k C_{A0} t}{1 + k C_{A0} t}$. Re-calculate the conversions for the PFR, CSTR, and non-ideal reactor. Is the impact of non-ideality more or less severe for a second-order reaction?
3.  **Mean Residence Time:** Use numerical integration (`quad`) to verify that the mean of the E-curve for the TIS model is indeed equal to $\tau$. The integral you need to solve is $\int_0^\infty t \cdot E(t) \, dt$.